# Focal Mechanisms and Moment Tensors

Focal mechanisms describe slip on a fault arising from a purely double couple source. See Stein and Wysession's Section 4.4 for a
discussion of force couples and single forces.Focal mechanisms inherently have ambiguity over which of the two resolved planes
is the actual fault plane, and in general additional evidence is required to determine which plane if the fault plane. Focal
mechanisms are commonly used to describe the faulting styles in a region, and to invert for principle stress axis orientations.
Understanding stress orientations (and magnitude, which cannot be derived from focal mechanism inversion) is becoming useful
for understanding the likelihood of induced earthquakes (e.g.: 
[Lund Snee and Zoback 2018](https://pubs.geoscienceworld.org/tle/article/37/2/127/527282/state-of-stress-in-the-permian-basin-texas-and-new))
and for understanding how stresses change around slow-slip events (e.g. [Warren-Smith et al., 2019]()) amongst other applications.

## First motions

The simplest and most widely used method for estimating focal mechanisms is to evaluate first-motion polarities of P-waves.
The `seismic_picker.py` applet has been extended to allow you to pick polarities. Note that this needs to be the first motion
polarity, not the polarity of the maximum peak. The wavetrain of an individual seismic phase is a superposition of different
ray-paths, which may be of opposite polarities.  Polarities are either compressional (up) or dilatational (down). Waves that
show compressive first motions arise when the faults first motion is towards the seismometer (along the ray-path, which may
take an *interesting* path to the seismometer). Dilatational waves arise when the faults first motion is away from the seismometer.

<img alt="Stein and Wysession Fig 4.2-4" align="centre" style="width:40%" src="http://levee.wustl.edu/seismology/book/chapter4/chap4_sr/4_2_04_s.jpg">

Once polarities are measured they can be plotted on a stereonet to obtain a scatter of points on the focal sphere. These points can
be inverted to obtain a best fitting focal mechanism. Some important things to note are:
1. First motion amplitudes tend to zero towards the nodal planes: polarities may be obscured close to nodal planes;
2. Polarities are plotted based on take-off angle and azimuth. Both properties require accurate hypocentre locations and
   accurate ray-tracing;
3. Lower-hemisphere stereonets are commonly used because take-off angles for first arriving rays are often downwards (towards
   a faster medium).
   
To make the stereonet plots below you will need to install mplstereonet:
```bash
pip install mplstereonet
```

In [1]:
%matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from utilities.seismic_picker import SeismicPicker
from utilities.plot_event import get_geonet_waveforms
from obspy.clients.fdsn import Client

from obspy import Stream
from obspy.core.event import ResourceIdentifier
from obspy.clients.fdsn import Client

event_id = "2018p546311"

client = Client("GEONET")
cat = client.get_events(eventid=event_id)
event = cat[0]
print("Event has {0} picks".format(len(event.picks)))
# We will remove the picks from strong motion sites, they often have poor timing
event.picks = [p for p in event.picks 
               if p.waveform_id.channel_code[0] != "B"]

# Lets just look at the first 20
event.picks = sorted(event.picks, key=lambda p: p.time)[0:20]

st = get_geonet_waveforms(event=event, all_components=True)

# We want to remove the arrival info because we are changing the picks
for origin in event.origins:
    origin.arrivals = []

# We will add polarities to this GeoNet event.
picked_event = SeismicPicker(st.select(component="Z"), event_in=event).pick()

# I wrote my picks out to "polarity_picked.xml": 
# picked_event.write("polarity_picked.xml", format="QUAKEML")

Using matplotlib backend: Qt5Agg
Event has 204 picks
Make your picks using:
	left mouse button: P
	right mouse button: S
	'a': amplitude at mouse location
	'e': end duration at mouse location - needs a P-pick to calculate duration
	P-wave polarity can be picked using the up and down arrows while hovering over a P-pick.

Picks can be deleted by hovering over them and pressing the middle mouse button
Finished picking.
Finished processing event. Returning
Returning event


We now need to locate the event with the new picks - this will calculate take-off angles and azimuths that
we can use for focal mechanism calculation.  Note that `event.origins[n].arrivals` contains this information,
and we could have used the GeoNet derived information, but we have changed the picks so the arrivals no longer
match the picks. Thus we have to locate again. We will also calculate the focal mechanism at the same time, using
the HASH software.

If you get an error like:
```bash
FileNotFoundError(hash was not here: ...)
```
then you will need to add the `seisan_path=<path to seisan>` to the call to `seisan_hash`, where `<path_to_seisan>` should be where you installed Seisan, for me it is `~/Desktop/Seisan`.

In [2]:
from obspy import read_events
from obspy.clients.fdsn import Client
from utilities.geiger_inversion import seisan_hash

client = Client("GEONET")

event = read_events("polarity_picked.xml")[0]
bulk = [
    (pick.waveform_id.network_code, pick.waveform_id.station_code,
     pick.waveform_id.location_code, pick.waveform_id.channel_code, 
     pick.time - 300, pick.time + 300) for pick in event.picks]
inventory = client.get_stations_bulk(bulk, level="channel")
velocities = [
    {"velocity": 6.2, "top": 0.0},
    {"velocity": 6.6, "top": 12.0},
    {"velocity": 7.1, "top": 23.0},
    {"velocity": 8.05, "top": 31.0, "moho": True},
    {"velocity": 8.25, "top": 50.0},
    {"velocity": 8.5, "top": 80.0}]

event_located = seisan_hash(event, inventory, velocities, vpvs=1.74)
event_located.preferred_origin_id = event_located.origins[-1].resource_id

C:\Users\Calum\Anaconda3\envs\gphs445\lib\site-packages\obspy\io\stationxml\core.py:98: UserWarning: The StationXML file has version 1, ObsPy can read versions (1.0, 1.1). Proceed with caution.
  version, ", ".join(READABLE_VERSIONS)))


  Max number of polarity errors, defualt is 0
  Max average error in amp rat, log10, def 0.1
  Enter angle for computing mechanisms probability, def is 60
  Enter probability threshold for multiples, def is 0.1


Number of polarities:           18
Amplitude types:   Manual:       0   Automatic:    0   Spectral:     0


 Q: Local: Qp= 470.0**0.70  Qs= 470.0** 0.7   Global: t*(P)=1.10  t*(S)=4.20

 STAT  C PH       AMP    PER TRTIME   QCOR ANGINC ANGEMG Fcor   AZ  DIST

 STAT  Ratio type  T     Amp 1    Amp 2  Fcor LogRat

 total obs =  18  gap in az =  86.0  gap in ain =  28.0
  Number of events           0
 vpvs ratio is   0.00
 vpvse set to 1.74
 Number of polarities is                      :    18
 *** warning - no s/p amplitude ratios for event
 Minimum number of polarity misfits overall   :     1
 New number of pol. misfits inc. extra is     :     2
 Number of solutions found                       2000

 Strike,dip,rake                 3.8    64.9   154.1
 Fault+aux plane uncertain

We can now try and plot this. Note that this is a deep event, so take-off angles are large (remember
that they are measured from radial towards the centre of the Earth).

In [4]:
from utilities.plot_event import plot_polarities

fig = plot_polarities(event_located)

## Amplitude ratios

A simple extension of the first motion method is to take advantage of the difference in P and S-wave radiation patterns:

<img alt="Stein and Wysession Fig 4.2-7" align="centre" style="width:20%" src="http://levee.wustl.edu/seismology/book/chapter4/chap4_sr/4_2_07_s.jpg">

By taking the ratio of P and S wave amplitudes we can estimate where on the focal sphere we are. This provides additional information
to better constrain the focal mechanism, and can be especially useful near the nodal planes, where S amplitude is maximized but first
motion polarities of P waves can be hard to pick. We won't make use of this in your assignment, but you should read Stein and Wysession 4.2.3.

Focal mechanisms can also be estimated by modelling body or/and surface waves, see Stein and Wysession 4.3.

## Moment tensor inversion

So far we have just considered the information relating to simple slip on a single fault, however, natural events are often more complicated. At relatively low sampling
frequencies and large station-reciever distances, the high frequencies that show this complexity are lost, however, for larger events the complexity is manifest at
lower frequencies, meaning that we can resolve some of these complexities using seismic data. One way to resolve some of the complexity is to compute moment tensors.

Read Stein and Wysession 4.4 on moment tensors.  For some information on how moment tensors are routinely computed for earthquakes in New Zealand,
read [Ristau 2008](https://pubs.geoscienceworld.org/ssa/srl/article/79/3/400/367690/implementation-of-routine-regional-moment-tensor).  The
moment tensors are available via the [GeoNet github page](https://github.com/GeoNet/data/tree/master/moment-tensor). You should select some
moment tensors from the GeoNet dataset to answer the "Slip Parameters" question in the assignment.